## Dependencies
* openai

In [5]:
import sys
sys.path.append("../")

In [6]:
import openai
from src.decorators import load_key_secret

In [19]:
path = "/home/propietari/Documents/claus/"
filename = "OpenAI_keys_JordiPlanas.json"
@load_key_secret(path, filename, "secret")
def return_secret(secret):
    return secret

def create_completion(text_to_complete, number_of_samples, tokens):
    return openai.Completion.create(
        engine = "davinci-instruct-beta",
        prompt = text_to_complete,
        temperature = 0.25,
        max_tokens = tokens,
        n = number_of_samples,
        echo = True
    )
def import_text_from_file(path, filename):
    file = path + filename
    with open(file) as f:
        lines = f.readlines()
    return lines

In [8]:
openai.api_key = return_secret()

In [20]:
path = "../input/"
filename = "Song.txt"
original_text = import_text_from_file(path, filename)

In [22]:
# response = openai.Completion.create(
#   engine="davinci-instruct-beta",
#   prompt="Artificial intelligence (AI) is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals, which involves consciousness and emotionality. The distinction between the former and the latter categories is often revealed by the acronym chosen. 'Strong' AI is usually labelled as AGI (Artificial General Intelligence) while attempts to emulate 'natural' intelligence have been called ABI (Artificial Biological Intelligence). Leading AI textbooks define the field as the study of \"intelligent agents\": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.[3] Colloquially, the term \"artificial intelligence\" is often used to describe machines (or computers) that mimic \"cognitive\" functions that humans associate with the human mind, such as \"learning\" and \"problem solving\".[4]\n\nAs machines become increasingly capable, tasks considered to require \"intelligence\" are often removed from the definition of AI, a phenomenon known as the AI effect.[5] A quip in Tesler's Theorem says \"AI is whatever hasn't been done yet.\"[6] For instance, optical character recognition is frequently excluded from things considered to be AI,[7] having become a routine technology.[8] Modern machine capabilities generally classified as AI include successfully understanding human speech,[9] competing at the highest level in strategic game systems (such as chess and Go),[10] autonomously operating cars, intelligent routing in content delivery networks, and military simulations.[11]\n\ntl;dr:",
#   temperature=0.25,
#   max_tokens=100,
#   top_p=1
# )
response = create_completion(original_text[0:4])

In [23]:
response

<OpenAIObject text_completion id=cmpl-2oUWJDMErTlgTBcvFZOoTQ9z0Awjj at 0x7f5cbf55bbd0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\nI'm sorry, I don't speak Australian"
    },
    {
      "finish_reason": "length",
      "index": 1,
      "logprobs": null,
      "text": "\nI don\u2019t know.\n\n"
    },
    {
      "finish_reason": "length",
      "index": 2,
      "logprobs": null,
      "text": "\nYes, there are days when I\u2019"
    },
    {
      "finish_reason": "length",
      "index": 3,
      "logprobs": null,
      "text": "\nI'm not sure\n\nI'm not"
    }
  ],
  "created": 1618386395,
  "id": "cmpl-2oUWJDMErTlgTBcvFZOoTQ9z0Awjj",
  "model": "if-davinci-v2",
  "object": "text_completion"
}

In [ ]:
summary_text = response["choices"][0]["text"].split("\n")[0]

In [ ]:
summary_text